# SPT analysis - Interactive report
###### AB @ BCB ITQB 


### Instructions:
1. To run a cell select it and press **SHIFT+ENTER**;
2. All cells require the **previous** one to be run first;
3. If you change the loaded file through the dropdown menu, **RUN** the interactive graph cell (the next one) again to update the data 
4. If you make a mistake in the order or just want to restart the report **restart the kernel** and start from the beginning
5. Reading a .npy file takes a few seconds depending on its size. The loading is cached, meaning that loading a previously loaded file is instant.
---

### Import required functions

In [1]:
%matplotlib widget
from ReportJupyter import *

### Choose folder that contains all npy's

In [2]:
# Choose folder
root = Tk()
rootdir = filedialog.askdirectory()
root.destroy()

load_widget = interactive(FileLoader, 
                          folder=Dropdown(options=os.listdir(rootdir), description='Experimental conditions:', disabled=False), 
                          root = fixed(rootdir))
display(load_widget)

out = Output()
button = Button(description="Click here to preload ALL data")
button.on_click(partial(loadall, rootdir, out))
display(button)
display(out)


interactive(children=(Dropdown(description='Experimental conditions:', options=('2021-11-15_divIB_30C', '2021-…

Button(description='Click here to preload ALL data', style=ButtonStyle())

Output()

In [46]:
def Update_Graphs(angle_threshold, major_threshold, all_tracks):
    
    plt.close('all')
    
    all_angles = np.rad2deg(np.arccos([i.ellipse['minor'] / i.ellipse['major'] for i in all_tracks]))
    all_diameter = np.array([i.ellipse['major']*1000 for i in all_tracks])
    
    filtered_tracks = []
    for idx,tr in enumerate(all_tracks):
        if (all_angles[idx]<angle_threshold[1] and all_angles[idx]>angle_threshold[0]) and (all_diameter[idx]<major_threshold[1] and all_diameter[idx]>major_threshold[0]):
            filtered_tracks.append(tr)
    filtered_tracks = np.array(filtered_tracks)
    
    print(
        f"n = {len(filtered_tracks)} (out of {len(all_tracks)} ({(len(filtered_tracks) / len(all_tracks)) * 100:.1f}%))\n\n")

    angles = np.rad2deg(np.arccos([i.ellipse['minor'] / i.ellipse['major'] for i in filtered_tracks]))
    diameter = np.array([i.ellipse['major']*1000 for i in filtered_tracks])
    
    displacement_velo = [np.average(tr.disp_velo) for tr in filtered_tracks]
    brute_velo = [tr.bruteforce_velo for tr in filtered_tracks]
    brute_sections = [len(tr.bruteforce_phi) for tr in filtered_tracks]
    mug_velo = [tr.muggeo_velo for tr in filtered_tracks]
    mug_sections = [len(tr.muggeo_phi) for tr in filtered_tracks]
    
    print(f"Displacement: {np.nanmean(displacement_velo):.2f} +- {np.nanstd(displacement_velo):.2f} nm/s \n")
    print(f"Brute force sectioning: {np.average(np.hstack(brute_velo)):.2f} +- {np.std(np.hstack(brute_velo)):.2f} nm/s")
    print(f"\t Average number of sections of {np.average(brute_sections):.2f} \n")
    print(f"Muggeo et al sectioning: {np.nanmean(np.hstack(mug_velo)):.2f} +- {np.nanstd(np.hstack(mug_velo)):.2f} nm/s")
    print(f"\t Average number of sections of {np.nanmean(mug_sections):.2f} \n")

    
    ##############################################################################################################################################
    fig = plt.figure("Velocity Distributions", figsize=(10,5))
    gs = GridSpec(1,2, figure=fig, width_ratios=(1,1))
    
    ax1 = fig.add_subplot(gs[0,0])
    ax1.hist(displacement_velo, label="Displacement", alpha=0.5, density=True, bins='auto')
    ax1.hist(np.hstack(brute_velo), label="Brute force", alpha=0.5, density=True, bins='auto')
    ax1.hist(np.hstack(mug_velo), label="Muggeo et al", alpha=0.5, density=True, bins='auto')
    ax1.set_ylabel("Probability Density Function")
    ax1.set_xlabel("Velocity (nm/s)")
    ax1.legend()
    
    ax2 = fig.add_subplot(gs[0,1])
    ax2.hist(displacement_velo, label="Displacement", alpha=0.5, density=True, bins='auto')
    ax2.hist(np.hstack(brute_velo), label="Brute force", alpha=0.5, density=True, bins='auto')
    ax2.hist(np.hstack(mug_velo), label="Muggeo et al", alpha=0.5, density=True, bins='auto')
    ax2.set_ylabel("Probability Density Function")
    ax2.set_xlabel("Velocity (nm/s)")
    ax2.set_xscale('log')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    ##############################################################################################################################################
    
    ##############################################################################################################################################
    fig2 = plt.figure("Effect of angle", figsize=(10,10))
    gs = GridSpec(2,2, figure=fig2, width_ratios=(1,1), height_ratios=(1,1))
    
    ax1 = fig2.add_subplot(gs[0,0])
    ax1.scatter(x=angles, y=displacement_velo, label="Displacement")
    ax1.scatter(x=angles, y=[np.average(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax1.scatter(x=angles, y=[np.average(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax1.set_ylabel("Average Velocity (nm/s)")
    ax1.set_xlabel("Angle (deg)")
    ax1.legend()
    
    ax2 = fig2.add_subplot(gs[0,1])
    ax2.scatter(x=angles, y=[np.std(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax2.scatter(x=angles, y=[np.std(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax2.set_ylabel("Standard deviation (nm/s)")
    ax2.set_xlabel("Angle (deg)")
    ax2.legend()
    
    ax3 = fig2.add_subplot(gs[1,0])
    ax3.scatter(x=angles, y=brute_sections, label="Brute force", c='k', marker='*')
    ax3.scatter(x=angles, y=mug_sections, label="Muggeo et al", c='r', marker='+')
    ax3.set_ylabel("Number of sections")
    ax3.set_xlabel("Angle (deg)")
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    ##############################################################################################################################################
    
    ##############################################################################################################################################
    fig3 = plt.figure("Effect of diameter", figsize=(10,10))
    gs = GridSpec(2,2, figure=fig3, width_ratios=(1,1), height_ratios=(1,1))
    
    ax1 = fig3.add_subplot(gs[0,0])
    ax1.scatter(x=diameter, y=displacement_velo, label="Displacement")
    ax1.scatter(x=diameter, y=[np.average(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax1.scatter(x=diameter, y=[np.average(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax1.set_ylabel("Average Velocity (nm/s)")
    ax1.set_xlabel("Diameter (nm)")
    ax1.legend()
    
    ax2 = fig3.add_subplot(gs[0,1])
    ax2.scatter(x=diameter, y=[np.std(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax2.scatter(x=diameter, y=[np.std(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax2.set_ylabel("Standard deviation (nm/s)")
    ax2.set_xlabel("Diameter (nm)")
    ax2.legend()
    
    ax3 = fig3.add_subplot(gs[1,0])
    ax3.scatter(x=diameter, y=brute_sections, label="Brute force", c='k', marker='*')
    ax3.scatter(x=diameter, y=mug_sections, label="Muggeo et al", c='r', marker='+')
    ax3.set_ylabel("Number of sections")
    ax3.set_xlabel("Diameter (nm)")
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    ##############################################################################################################################################

    ##############################################################################################################################################
    fig4 = plt.figure("Effect of track length", figsize=(10,10))
    gs = GridSpec(2,2, figure=fig4, width_ratios=(1,1), height_ratios=(1,1))
    
    ax1 = fig4.add_subplot(gs[0,0])
    ax1.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=displacement_velo, label="Displacement")
    ax1.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=[np.average(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax1.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=[np.average(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax1.set_xlabel("Length of track (#)")
    ax1.set_ylabel("Velocity (nm/s)")
    ax1.legend()
    
    ax2 = fig4.add_subplot(gs[0,1])
    ax2.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=[np.std(i) for i in brute_velo], label="Brute force", c='k', marker='*')
    ax2.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=[np.std(i) for i in mug_velo], label="Muggeo et al", c='r', marker='+')
    ax2.set_xlabel("Length of track (#)")
    ax2.set_ylabel("Standard deviation (nm/s)")
    ax2.legend()
    
    ax3 = fig4.add_subplot(gs[1,0])
    ax3.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=brute_sections, label="Brute force", c='k', marker='*')
    ax3.scatter(x=[len(tr.unwrapped) for tr in filtered_tracks], y=mug_sections, label="Muggeo et al", c='r', marker='+')
    ax3.set_xlabel("Length of track (#)")
    ax3.set_ylabel("Number of sections")
    ax3.legend()
    
    plt.tight_layout()
    plt.show()
    ##############################################################################################################################################


### Interactive filtering
- Filter the tracks by assigning a maximum allowed angle;
- This angle refers to the deviation of the Z-ring ellipse to a perfect circle
- Higher angles correspond to slanted rings and higher errors

In [47]:
TrackObjects = np.array(load_widget.result)

#diameter = np.array([tr.ellipse['major']*1000 for tr in TrackObjects])
#angle = np.rad2deg(np.arccos([i.ellipse['minor']/i.ellipse['major'] for i in TrackObjects]))

angle_slider = IntRangeSlider(value=[0, 60], min=0, max=90, step=1, description='Angle:')
major_slider = IntRangeSlider(value=[0, 800], min=0, max=2000, step=10, description='Diameter:')

angle_wid = interactive(Update_Graphs, angle_threshold=angle_slider, major_threshold=major_slider,
            all_tracks=fixed(TrackObjects))

angle_wid


interactive(children=(IntRangeSlider(value=(0, 60), description='Angle:', max=90), IntRangeSlider(value=(0, 80…

### Interactive comparison
- After defining appropriate filtering in the above cells select tracks for comparison
- By default, adding a ftsW conditions WILL look for the corresponding divIB

In [12]:
selections = SelectMultiple(
    options=os.listdir(rootdir),
    description='Available Tracks',
    rows=len(os.listdir(rootdir)),
    width=200,
    disabled=False
)



angle = BoundedIntText(
    value=90,
    min=0,
    max=91,
    step=1,
    description='Angle Threshold:',
    disabled=False
)

comparison_wid = interactive(ViolinComparison, conditions=selections, root=fixed(rootdir), anglethresh=angle)
comparison_wid

interactive(children=(SelectMultiple(description='Available Tracks', options=('2021-11-15_divIB_30C', '2021-11…